# Carga de Datos - California Housing Dataset

Este notebook se encarga de la carga inicial de datos desde el archivo CSV.

**Objetivo**: Cargar el dataset y realizar una exploración inicial básica.

**Dataset**: California Housing - Predicción de precios de viviendas en California

**Autor**: MLOps Pipeline Project  
**Fecha**: Noviembre 2025

## 1. Importar Librerías Necesarias

In [ ]:
import pandas as pd
import numpy as np
import json
import warnings
warnings.filterwarnings('ignore')

# Para visualización básica
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de visualización
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

print("✓ Librerías importadas exitosamente")
print(f"Versión de pandas: {pd.__version__}")
print(f"Versión de numpy: {np.__version__}")

## 2. Cargar Configuración del Proyecto

In [ ]:
# Cargar configuración desde config.json
with open('../config.json', 'r') as f:
    config = json.load(f)

print("=" * 80)
print("CONFIGURACIÓN DEL PROYECTO")
print("=" * 80)
for key, value in config.items():
    print(f"{key:20s}: {value}")
print("=" * 80)

## 3. Cargar el Dataset

In [ ]:
# Cargar el dataset desde el archivo CSV
data_path = f"../{config['data_file']}"
df = pd.read_csv(data_path)

print("=" * 80)
print("DATASET CARGADO EXITOSAMENTE")
print("=" * 80)
print(f"Archivo: {config['data_file']}")
print(f"Dimensiones: {df.shape[0]:,} filas × {df.shape[1]} columnas")
print(f"Tamaño en memoria: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")
print("=" * 80)

## 4. Exploración Inicial de Datos

In [ ]:
# Primeras filas del dataset
print("=" * 80)
print("PRIMERAS 10 FILAS DEL DATASET")
print("=" * 80)
df.head(10)

In [ ]:
# Información general del dataset
print("=" * 80)
print("INFORMACIÓN GENERAL DEL DATASET")
print("=" * 80)
df.info()

### 4.1 Descripción de las Columnas

In [ ]:
print("=" * 80)
print("DESCRIPCIÓN DE LAS COLUMNAS DEL DATASET")
print("=" * 80)
print("\nColumnas disponibles:")
for i, col in enumerate(df.columns, 1):
    print(f"  {i:2d}. {col}")

print("\n" + "=" * 80)
print("DICCIONARIO DE DATOS")
print("=" * 80)

column_descriptions = {
    'longitude': 'Longitud geográfica (coordenada oeste-este)',
    'latitude': 'Latitud geográfica (coordenada norte-sur)',
    'housing_median_age': 'Edad mediana de las viviendas en el bloque',
    'total_rooms': 'Número total de habitaciones en el bloque',
    'total_bedrooms': 'Número total de dormitorios en el bloque',
    'population': 'Población total del bloque',
    'households': 'Número total de hogares/familias en el bloque',
    'median_income': 'Ingreso mediano de los hogares (en decenas de miles de USD)',
    'ocean_proximity': 'Proximidad al océano (categórica)',
    'median_house_value': '🎯 VARIABLE OBJETIVO: Valor mediano de la vivienda (USD)'
}

for col, desc in column_descriptions.items():
    print(f"\n{col}:")
    print(f"  {desc}")

## 5. Valores Nulos y Tipos de Datos

In [ ]:
# Análisis de valores nulos
print("=" * 80)
print("ANÁLISIS DE VALORES NULOS")
print("=" * 80)

null_counts = df.isnull().sum()
null_percentage = (df.isnull().sum() / len(df)) * 100

null_df = pd.DataFrame({
    'Columna': null_counts.index,
    'Valores Nulos': null_counts.values,
    'Porcentaje (%)': null_percentage.values,
    'Tipo de Dato': df.dtypes.values
}).sort_values('Valores Nulos', ascending=False)

print(null_df.to_string(index=False))

total_nulls = df.isnull().sum().sum()
total_cells = df.shape[0] * df.shape[1]
print(f"\n{'='*80}")
print(f"Total de valores nulos: {total_nulls:,} de {total_cells:,} celdas ({total_nulls/total_cells*100:.2f}%)")
print("=" * 80)

### 5.1 Clasificación de Variables

In [ ]:
# Clasificación de variables
print("=" * 80)
print("CLASIFICACIÓN DE VARIABLES")
print("=" * 80)

numeric_cols = df.select_dtypes(include=['int64', 'float64']).columns.tolist()
categorical_cols = df.select_dtypes(include=['object']).columns.tolist()

print(f"\n📊 Variables Numéricas ({len(numeric_cols)}):")
for col in numeric_cols:
    print(f"  • {col}")

print(f"\n📁 Variables Categóricas ({len(categorical_cols)}):")
for col in categorical_cols:
    unique_values = df[col].nunique()
    print(f"  • {col} ({unique_values} valores únicos)")
    
print("\n" + "=" * 80)

## 6. Estadísticas Descriptivas Básicas

In [ ]:
# Estadísticas descriptivas para variables numéricas
print("=" * 80)
print("ESTADÍSTICAS DESCRIPTIVAS - VARIABLES NUMÉRICAS")
print("=" * 80)
df.describe().T

In [ ]:
# Estadísticas para variables categóricas
print("=" * 80)
print("ESTADÍSTICAS DESCRIPTIVAS - VARIABLES CATEGÓRICAS")
print("=" * 80)

for col in categorical_cols:
    print(f"\n{col}:")
    print(f"  Valores únicos: {df[col].nunique()}")
    print(f"  Distribución:")
    value_counts = df[col].value_counts()
    for val, count in value_counts.items():
        percentage = (count / len(df)) * 100
        print(f"    • {val}: {count:,} ({percentage:.1f}%)")

## 7. Visualización Inicial

In [ ]:
# Visualización de la distribución de la variable objetivo
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Histograma
axes[0].hist(df['median_house_value'], bins=50, edgecolor='black', alpha=0.7)
axes[0].set_xlabel('Median House Value (USD)')
axes[0].set_ylabel('Frequency')
axes[0].set_title('Distribución de Precios de Viviendas')
axes[0].grid(True, alpha=0.3)

# Boxplot
axes[1].boxplot(df['median_house_value'])
axes[1].set_ylabel('Median House Value (USD)')
axes[1].set_title('Boxplot de Precios de Viviendas')
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print(f"\nVariable Objetivo: median_house_value")
print(f"  Mínimo: ${df['median_house_value'].min():,.2f}")
print(f"  Máximo: ${df['median_house_value'].max():,.2f}")
print(f"  Media: ${df['median_house_value'].mean():,.2f}")
print(f"  Mediana: ${df['median_house_value'].median():,.2f}")

In [ ]:
# Visualización geográfica de los datos
plt.figure(figsize=(12, 8))
scatter = plt.scatter(df['longitude'], df['latitude'], 
                     c=df['median_house_value'], 
                     cmap='viridis', 
                     alpha=0.4, 
                     s=10)
plt.colorbar(scatter, label='Median House Value (USD)')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.title('Distribución Geográfica de Precios de Viviendas en California')
plt.grid(True, alpha=0.3)
plt.show()

## 8. Resumen de la Carga de Datos

In [ ]:
print("=" * 80)
print("RESUMEN DE LA CARGA DE DATOS")
print("=" * 80)
print(f"\n✓ Dataset cargado exitosamente desde: {config['data_file']}")
print(f"✓ Total de registros: {df.shape[0]:,}")
print(f"✓ Total de características: {df.shape[1]}")
print(f"✓ Variables numéricas: {len(numeric_cols)}")
print(f"✓ Variables categóricas: {len(categorical_cols)}")
print(f"✓ Valores nulos encontrados: {total_nulls:,} ({total_nulls/total_cells*100:.2f}%)")
print(f"✓ Tamaño en memoria: {df.memory_usage(deep=True).sum() / 1024**2:.2f} MB")

print(f"\n📌 Observaciones Iniciales:")
print(f"  • La variable 'total_bedrooms' tiene {df['total_bedrooms'].isnull().sum():,} valores nulos")
print(f"  • Hay {df['ocean_proximity'].nunique()} categorías en 'ocean_proximity'")
print(f"  • Los precios varían desde ${df['median_house_value'].min():,.0f} hasta ${df['median_house_value'].max():,.0f}")
print(f"  • Los datos están distribuidos geográficamente por toda California")

print(f"\n🎯 Variable Objetivo: median_house_value")
print(f"\n✅ Datos listos para el análisis exploratorio (EDA)")
print("=" * 80)